# This Jupyter Notebook goes through the different development stages of the Python Fullstack Assessment Project for Stori

## First steps:

* ### Work around Pythonic Email Delivery
* ### Work around CSV file and other data managment to create each message

## Secondary steps:

* ### Work around dockerization
* ### Work around extending the system to more than one users
* ### Work around AWS DevOps

## Pythonic Email Delivery

### Using smtplib, SSL and Gmail, emails will be sent to a given account
### The authenticaton credentials for the sender will be stored in a pickle file, which only has to be created once. 

## Pickle File for authentication credentials

### Fill the credentials dictionary, run the code once, and then erase the entries.

### This code will create a ```.pkl``` file which will contain the mailing credentials. If the file already exists, then it will load it. 

### This way, sharing code is possible without revealing personal credentials;  ```.gitignore``` already includes this ```.pkl``` file, so it's not passed onto the GitHUB repo. 

In [1]:
import pickle
import os
if not os.path.exists('secret_credentials.pkl'):
    credentials={}
    credentials['Sender Email'] = "" # sender email account
    credentials['Sender Password'] = "" # sender email account password
    credentials['Receiver Email'] = "" # receiver email account
    with open('secret_credentials.pkl','wb') as f:
        pickle.dump(credentials, f)
else:
    credentials=pickle.load(open('secret_credentials.pkl','rb'))

## Mailing Tools and Generic Email Body.

![email_1try.png](email_1try.png)

## Add Stori Logo:

### Follow Signarture and Logo suggestions from [Exclaimer](https://www.exclaimer.com/email-signature-handbook/using-the-right-email-signature-size/#:~:text=Email%20signature%20size%20dimensions,70%20to%20100%20pixels%20high.)

![stori.jpg](stori.jpg)
                                                                                                            

In [7]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

port = 465 # Assign 465 port for the SMTP server (Gmail's requirements)
context = ssl.create_default_context() 

In [16]:
sender_email = credentials['Sender Email']
password =credentials['Sender Password']
receiver_email = credentials['Receiver Email'] # receiver email account

# create Multipurpose Internet Mail Extensions object
message = MIMEMultipart("alternative")
message["Subject"] = "Transactions Summary"
message["From"] = sender_email
message["To"] = receiver_email

# If running cells in order, uncomment the following lines to avoid variable errors.
# total_balance = -1
# avg_debit = -1
# avg_credit = -1

# monthly_summary = "tbd"

html = f"""
<html>
    <body>
        <div>
            <p>
                Hello,<br>Herein is your Transactions Summary<br>
            </p>
        </div>
        <div>
            <table style="width:100%">
                  <tr>
                    <th>
                        <p>
                            <br>Total Balance: {total_balance}<br>
                            <br>Average Debit Amount: {avg_debit}<br>
                            <br>Average Credit Amount: {avg_credit}<br>
                        </p>
                    </th>
                    <th>
                        <p>
                            {monthly_summary}
                        </p>
                    </th>
                  </tr>
            </table>
        </div>  
        <div>
            <br>
                <img align="left" width="300" height="100" src="cid:image1">
            <br>
        </div>
    </body>
        
</html>
"""
html_text = MIMEText(html, "html")
message.attach(html_text)

fp = open('stori.jpg', 'rb')
msgImage = MIMEImage(fp.read())
fp.close()

msgImage.add_header('Content-ID', '<image1>')
message.attach(msgImage)

with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )

## CSV file managment

### Using pandas and its built-in datetime utilities, the ```.csv``` files can be handled and processed easily. 

In [3]:
import pandas as pd
# load trxn.csv
trxns = pd.read_csv("trxn.csv")

trxns

,id,date,transaction
0,0,10/1,10
1,1,15/1,-15
2,2,10/2,20
3,3,15/2,-25
4,4,10/3,30
5,5,15/3,-35


In [4]:

#Total Balance
total_balance = trxns.transaction.sum()
print(f"Total Balance is {total_balance}")

#Debit and Credit Averages
trxns["DebitorCredit"] = trxns.transaction.map( lambda x: 'credit' if x > 0 else 'debit')
avg_credit = trxns.loc[trxns.DebitorCredit == "credit", 'transaction'].mean()
avg_debit = trxns.loc[trxns.DebitorCredit == "debit", 'transaction'].mean()
print(f"Average Credit is {avg_credit}")
print(f"Average Debit is {avg_debit}")

#Monthly Summaries
trxns.date = pd.to_datetime(trxns.date, format = "%d/%m")
trxns['month'] = trxns.date.dt.strftime('%B')
trx_per_month = trxns.month.value_counts()
monthly_summary = ""
for month, count in zip(trx_per_month.index, trx_per_month):
    monthly_summary += f"<br>Number of transactions {month}: {count}<br>\n"
print("String to be injected inside the email:\n'", monthly_summary)

trxns

Total Balance is -15
Average Credit is 20.0
Average Debit is -25.0
String to be injected inside the email:
' <br>Number of transactions January: 2<br>
<br>Number of transactions February: 2<br>
<br>Number of transactions March: 2<br>



,id,date,transaction,DebitorCredit,month
0,0,1900-01-10,10,credit,January
1,1,1900-01-15,-15,debit,January
2,2,1900-02-10,20,credit,February
3,3,1900-02-15,-25,debit,February
4,4,1900-03-10,30,credit,March
5,5,1900-03-15,-35,debit,March
